# DeepSeek-TS+: A Unified Framework for Multi-Product Time Series Forecasting
This notebook provides a detailed explanation of DeepSeek-TS+, a unified framework for multi-product time series forecasting. It covers the theoretical foundations, architecture, and a real-life application example.

## 1. Introduction to Multi-Product Time Series Forecasting
Multi-product time series forecasting involves predicting future values for multiple related time series simultaneously. This is common in various business domains, such as retail (sales forecasting for different products), finance (stock prices for multiple assets), and supply chain (demand forecasting for various components). Traditional methods often treat each time series independently, which can lead to suboptimal results as they fail to capture inter-series dependencies.

## 2. Challenges in Multi-Product Time Series Forecasting
- **Inter-series Dependencies:** Products often influence each other (e.g., complementary or substitute goods). Ignoring these relationships can lead to inaccurate forecasts.
- **Data Heterogeneity:** Different products may have varying scales, seasonality, and trends.
- **Sparsity:** Some products might have infrequent sales or missing data, making it challenging to train robust models.
- **Scalability:** Managing and forecasting thousands or millions of time series efficiently is a significant challenge.

## 3. DeepSeek-TS+ Framework
DeepSeek-TS+ addresses these challenges by providing a unified framework that leverages deep learning to capture complex patterns and dependencies across multiple time series. It aims to improve forecasting accuracy and efficiency by jointly modeling all series.

### 3.1. Core Components
While the exact architecture of DeepSeek-TS+ would require a deep dive into the original paper, a unified framework typically includes:
- **Shared Feature Extraction:** A common neural network backbone to extract relevant features from all time series, allowing the model to learn shared patterns.
- **Inter-series Interaction Module:** Mechanisms (e.g., attention mechanisms, graph neural networks) to explicitly model dependencies between different time series.
- **Product-Specific Heads:** Output layers tailored for each product to generate individual forecasts, while still benefiting from the shared representations.

### 3.2. Advantages
- **Improved Accuracy:** By modeling inter-series dependencies, DeepSeek-TS+ can achieve higher forecasting accuracy compared to independent models.
- **Enhanced Robustness:** Shared learning can make the model more robust to data sparsity or noise in individual series.
- **Scalability:** A unified framework can be more efficient to train and deploy for a large number of products.

## 4. Real-Life Application: Retail Sales Forecasting
Let's consider a real-life application of DeepSeek-TS+ for retail sales forecasting across a large number of products in a supermarket chain. The goal is to accurately predict daily sales for thousands of unique products, considering factors like promotions, holidays, and inter-product relationships.

### 4.1. Problem Definition
- **Input:** Historical daily sales data for N products, promotional calendars, holiday information, and potentially external factors like weather or economic indicators.
- **Output:** Forecasted daily sales for each of the N products for the next M days.
- **Challenge:** Capturing the complex interplay between product sales (e.g., increased sales of hot dog buns when hot dogs are on promotion), seasonality, and external events.

### 4.2. Data Preparation (Conceptual)
For a real-world scenario, data would need to be collected, cleaned, and preprocessed. This would involve:
- **Time Series Alignment:** Ensuring all product sales data are aligned by date.
- **Feature Engineering:** Creating features like day of week, month, year, holiday flags, promotional flags, and potentially lagged sales data.
- **Normalization:** Scaling sales data to a common range to improve neural network training.

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Conceptual Data Generation (for demonstration purposes)
np.random.seed(42)
num_products = 5
num_days = 365 * 2
dates = pd.date_range(start=\'2023-01-01\', periods=num_days, freq=\'D\')

data = {f\'product_{i+1}\' : np.random.rand(num_days) * 100 + 50 + 20 * np.sin(np.arange(num_days) / 30) for i in range(num_products)}
df = pd.DataFrame(data, index=dates)

# Introduce some inter-product dependency (e.g., product 1 influences product 2)
df[\'product_2\'] = df[\'product_2\'] + 0.5 * df[\'product_1\'].shift(7).fillna(0)

# Add a promotional effect for product 3
promo_days = np.random.choice(num_days, 20, replace=False)
df.loc[df.index[promo_days], \'product_3\'] = df.loc[df.index[promo_days], \'product_3\'] * 1.5

print("Sample Data:")
print(df.head())

### 4.3. Model Architecture (Conceptual DeepSeek-TS+ inspired)
A simplified conceptual model inspired by unified time series forecasting frameworks. This example uses a shared LSTM layer to capture temporal dependencies and then separate linear layers for each product.

In [ ]:
class DeepSeekTSModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dims):
        super(DeepSeekTSModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.product_heads = nn.ModuleList([nn.Linear(hidden_dim, out_dim) for out_dim in output_dims])

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        lstm_out, _ = self.lstm(x)
        # Take the output from the last time step
        last_time_step_output = lstm_out[:, -1, :]
        
        outputs = [head(last_time_step_output) for head in self.product_heads]
        return torch.cat(outputs, dim=1) # Concatenate forecasts for all products

# Prepare data for training
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 30 # Use 30 days of historical data to predict the next day
X, y = create_sequences(scaled_data, seq_length)

X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Split data into training and testing
train_size = int(len(X_tensor) * 0.8)
X_train, X_test = X_tensor[:train_size], X_tensor[train_size:]
y_train, y_test = y_tensor[:train_size], y_tensor[train_size:]

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Model instantiation
input_dim = num_products # Each product is a feature at each time step
hidden_dim = 64
num_layers = 2
output_dims = [1] * num_products # Predict 1 value for each product
model = DeepSeekTSModel(input_dim, hidden_dim, num_layers, output_dims)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f\'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}\')

# Evaluation
model.eval()
test_predictions = []
actuals = []
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        test_predictions.append(outputs.numpy())
        actuals.append(batch_y.numpy())

test_predictions = np.vstack(test_predictions)
actuals = np.vstack(actuals)

# Inverse transform to original scale
dummy_array_pred = np.zeros((len(test_predictions), num_products))
dummy_array_pred[:, :num_products] = test_predictions
original_predictions = scaler.inverse_transform(dummy_array_pred)[:, :num_products]

dummy_array_actual = np.zeros((len(actuals), num_products))
dummy_array_actual[:, :num_products] = actuals
original_actuals = scaler.inverse_transform(dummy_array_actual)[:, :num_products]

# Plotting results for one product (e.g., product 1)
plt.figure(figsize=(12, 6))
plt.plot(original_actuals[:, 0], label=\'Actual Product 1 Sales\')
plt.plot(original_predictions[:, 0], label=\'Predicted Product 1 Sales\')
plt.title(\'DeepSeek-TS+ Inspired Model: Product 1 Sales Forecasting\')
plt.xlabel(\'Time (Days)\')
plt.ylabel(\'Sales\')
plt.legend()
plt.show()

print("Mean Absolute Error for Product 1:", np.mean(np.abs(original_predictions[:, 0] - original_actuals[:, 0])))

## 5. Conclusion
This notebook provided a conceptual overview and a simplified implementation of a unified framework inspired by DeepSeek-TS+ for multi-product time series forecasting. By jointly modeling multiple series, such frameworks can capture complex dependencies and improve forecasting accuracy, which is crucial for business applications like retail sales planning.